<a href="https://colab.research.google.com/github/hank199599/deep_learning_keras_log/blob/main/Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用LSTM產生文字資料

## 生成序列資料
訓練神經網路為先前輸入的序列資料產生接續的資料，  
該神經網路被稱為語言模型，他會去捕捉語言的淺在空間(latent space)。  
* 條件資料(conditioning data)：初始的文字字串

## 字元等級的語言模型 (charactor-level neural language model)
使用LSTM層，從文字資料庫(text corpus)中萃取N個字元的字串作為輸入，  
並訓練模型預設第N+1個字元。  
  
**模型的輸出**：所有可能的字元經softmax運算後的結果  
![pic 8-1](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/8-1.png)

## 取樣策略的重要性
1. **greedy sampling** (貪婪取樣)：在所有可能字元中選擇最可能的下一個字元，但易導致重複的預測字串
2. **stochastic sampling** (隨機取樣): 自下一個字元的機率分布中取樣，並導入隨機性

### 控制隨機性 control the amount of randomness
在極端情形下，若分布不具隨機性會使抽樣結果則使該方法無效化
* 分布機率具最大熵(entorpy)：每個字元具有相同的可能性並具有最大的隨機性
* 分布機率具最小熵(entorpy)：字元分布幾乎不具備隨機性

#### softmax temperature：取樣機率分布熵的指標
代表下一個字元的不確定性

##### 範例
三個可能會出現的字元 a,b,c  
a 出現機率為 80%  
b,c 出現機率為 20%   
  
建立一個function針對不同的temperature設定，對機率分布重新加權並計算出新的機率分布

In [1]:
import numpy as np

def reweight_distribution(original_distribution,temperature=0.5):
  distribution = np.log(original_distribution)/temperature
  distribution = np.exp(distribution)
  return distribution / np.sum(distribution) #傳回重新加權
  
  # 因為重新加權後，機率分布之總和可能不再是1。因此再將其除以總合以滿足總合為1

In [2]:
ori_dstri = np.array([0.8,0.1,0.1])

In [3]:
new_dstri = reweight_distribution(ori_dstri,temperature=0.01)
print(new_dstri)

[1.00000000e+00 4.90909347e-91 4.90909347e-91]


In [4]:
new_dstri = reweight_distribution(ori_dstri,temperature=2)
print(new_dstri)

[0.58578644 0.20710678 0.20710678]


In [5]:
new_dstri = reweight_distribution(ori_dstri,temperature=10)
print(new_dstri)

[0.38102426 0.30948787 0.30948787]


![pic 8-2](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/8-2.png)  
使用較高temperature使亂度上升，產生更高的隨機性  
相反地，越低的溫度會使廳選出的樣本偏向特定一方

## 實現字元級LSTM文字資料生成 (本機端上操作)

In [6]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path).read().lower()
print('Corpus length:',len(text))

606208/600901 [==============================] - 0s 1us/step
Corpus length: 600893


以每3個字元作為間隔，萃取出一段段長度為maxlen的部分重疊序列資料

### 向量化字元序列

In [11]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0,len(text)-maxlen,step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])
print('Number of sequences:',len(sentences))

chars = sorted(list(set(text)))
print('Unique charactors:',len(chars))

# 將各個字元對應到"chars"串列中的索引值，成為字典格式
char_indices = dict((char,char.index(char)) for char in chars)

# 將字元經One-Hot編碼為二元陣列
print('Vectorization....')
x = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)

for i,sentence in enumerate(sentences):
  for t , char in enumerate(chars):
    x[i,t,char_indices[char]] = 1
  y[i,char_indices[next_chars[i]]] = 1

print(x.shape)
print(y.shape)


Number of sequences: 200278
Unique charactors: 57
Vectorization....
(200278, 60, 57)
(200278, 57)


# DeepDream

# 神經風格轉換

# 使用變分自編碼器 Variational Autoencoders 生成圖像

# 生成對抗神經網路簡介 Generative Adversarial Network
